In [1]:
import sqlite3
import pandas as pd
import os

# Connect to the database
db_path = 'opensubs.db'
if not os.path.exists(db_path):
    print(f"Error: Database file '{db_path}' not found.")
else:
    conn = sqlite3.connect(db_path)
    
    # Get database tables
    tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql_query(tables_query, conn)
    
    print(f"Database size: {os.path.getsize(db_path) / (1024**3):.2f} GB")
    print(f"\nTables in database:")
    print(tables)
    
    # For each table, show schema and sample data
    for table_name in tables['name']:
        print(f"\n\nTable: {table_name}")
        
        # Show table schema
        schema_query = f"PRAGMA table_info({table_name});"
        schema = pd.read_sql_query(schema_query, conn)
        print("\nSchema:")
        print(schema)
        
        # Show sample data (first 5 rows)
        sample_query = f"SELECT * FROM {table_name} LIMIT 5;"
        try:
            sample = pd.read_sql_query(sample_query, conn)
            print("\nSample data:")
            print(sample)
        except Exception as e:
            print(f"Error fetching sample: {e}")
    
    conn.close()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Database size: 127.42 GB

Tables in database:
   name
0  subz


Table: subz

Schema:
   cid  name     type  notnull dflt_value  pk
0    0   num  INTEGER        0       None   1
1    1  name     TEXT        0       None   0
2    2  file     BLOB        0       None   0

Sample data:
   num                                               name  \
0    1  attachment; filename="alien.3.(1992).eng.2cd.(...   
1    2  attachment; filename="identity.(2003).slv.1cd....   
2    3  attachment; filename="ghost.in.the.shell.2.inn...   
3    4  attachment; filename="planet.of.the.apes.(2001...   
4    5  attachment; filename="the.city.of.lost.childre...   

                                                file  
0  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x9a\x0e\x...  
1  b'PK\x03\x04\x14\x00\x00\x00\x08\x00m\x10\xf1T...  
2  b'PK\x03\x04\x14\x00\x00\x00\x08\x00w\x10\xf1T...  
3  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x84\x10\x...  
4  b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x8e\x10\x...  


In [3]:
import sqlite3

conn = sqlite3.connect("opensubs.db")
cursor = conn.cursor()

# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in database:", tables)

# Inspect columns for a specific table (e.g., subtitles)
cursor.execute("PRAGMA table_info(subtitles);")
columns = cursor.fetchall()
print("Subtitle table columns:", columns)

conn.close()


Tables in database: [('subz',)]
Subtitle table columns: []


In [5]:
# Sample from subtitles table
conn = sqlite3.connect("opensubs.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM subz LIMIT 10;")
sample_rows = cursor.fetchall()

for row in sample_rows:
    print(row)

conn.close()


(1, 'attachment; filename="alien.3.(1992).eng.2cd.(1).zip"', b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x9a\x0e\xf1T\x08zK{OH\x00\x00\xb6\xa0\x00\x00\x13\x00\x00\x00Alien.3.ENG.CD1.sub\x8d\xbdK\x93\xe3\xc6\x92%\xbc\x1f\xb3\xf9\x0fPmj\xc3J#\x89\xf7\xaa\xac\xf4\xaen\xbdZU\xd62};\x90\x04\x93P\x92\x04\x1b\x00\x8b\xca+\xf0\xbf\xcf9\xc7=@fI\xf7\x9b\x19\xeb\xb9(%#\x02\x81\x08\x0f\x7f\x1e\xf7\xf8kY&\xc9\xf5\xafx\x9e\xc7\xd7\x0fC\xd57}\xd4\x1c\x87\xba\xeb\xce\xa7\xa1\xde<\x8c\xdf6]\x8d\xbfD\xeb\xee\xb9}\xac\x8f\xcd:Z\xb7\x87S\xd5\r\x87\xfa8<\xfc\xef\xff\xc5\x9e\x19\xfa/\x8a\xf8\xfak}\xaa\xab\xe1\xff\xa5\xcf\xa2`\x9fe\x9a^\xdf\xed\xf7\xd1\xa9\xee\xfa\xf6x\xac\xf7QW\x9f\xdan\x88\x86\xf6\xe1A\xed\x96i\x81vq\\^\xebC\xdda\xac\xf5sT\xf7\xeb\xeaTG\x9f\xea]\xb3\xde\xd7\xd1\xbe:\x1f\xd7\xbbS\xb5Q\x878Y\xa2C\x82\x0e_\xd7\xf5)\xeaO\xd5\xba\x8e\xb6\xfb\xe6q7D\x97\x06\xef\xc2L0\x8bu\xed/H\xd4>\xc5\x12`\xbe\xaf>\xbe\x8a\x0e\xcd\xf1\xdcG\xcby\xd4\xd7\xeb\xf6\xb8\xe9\xd9\xaa\\.\xe3+\xfe\xb7,\xae\xef\x8e\xd17\x0f\xdf

In [8]:
import sqlite3
import zipfile
import io

# Connect to database
conn = sqlite3.connect("opensubs.db")
cursor = conn.cursor()

# Select a sample row (or multiple rows)
cursor.execute("SELECT filename, filedata FROM subz LIMIT 1;")
row = cursor.fetchone()

filename, filedata = row

# Use io.BytesIO to read blob data as a file-like object
with zipfile.ZipFile(io.BytesIO(filedata)) as z:
    # List all files inside the zip archive
    print("Files inside zip archive:", z.namelist())

    # Extract the content of each subtitle file
    for sub_filename in z.namelist():
        with z.open(sub_filename) as sub_file:
            subtitle_content = sub_file.read().decode('iso-8859-1')  # ISO-8859-1 or utf-8 depending on encoding
            print(f"\nSubtitle file: {sub_filename}\nContent preview:\n")
            print(subtitle_content[:1000])  # Print the first 1000 characters for preview

conn.close()


OperationalError: no such column: filename

In [1]:
import sqlite3

db_path = "opensubs.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(f"- {table[0]}")

# Get schema for the 'subz' table (and others if found)
print("\nSchema for 'subz' table:")
cursor.execute("PRAGMA table_info(subz);")
schema = cursor.fetchall()
# schema format: (column_id, column_name, data_type, not_null, default_value, primary_key)
for col in schema:
    print(f"- Name: {col[1]}, Type: {col[2]}, PK: {col[5]}")

conn.close()

Tables in the database:
- subz

Schema for 'subz' table:
- Name: num, Type: INTEGER, PK: 1
- Name: name, Type: TEXT, PK: 0
- Name: file, Type: BLOB, PK: 0


In [3]:
import gzip

metadata_path = "subtitles_all.txt.gz"
num_lines_to_check = 60 # Look a bit beyond line 55

print(f"--- First {num_lines_to_check} lines of {metadata_path} ---")
try:
    with gzip.open(metadata_path, 'rt', encoding='utf-8') as f: # Specify encoding if known, utf-8 is common
        for i, line in enumerate(f):
            print(f"Line {i+1}: {line.strip()}")
            if i >= num_lines_to_check - 1:
                break
except Exception as e:
    print(f"Error reading file: {e}")
    # Try a different encoding if utf-8 fails, e.g., 'latin-1' or 'cp1252'
    # try:
    #     with gzip.open(metadata_path, 'rt', encoding='latin-1') as f:
    #         # ... read lines ...
    # except Exception as e2:
    #      print(f"Also failed with latin-1: {e2}")

print("--- End of file sample ---")

--- First 60 lines of subtitles_all.txt.gz ---
Line 1: IDSubtitle	MovieName	MovieYear	LanguageName	ISO639	SubAddDate	ImdbID	SubFormat	SubSumCD	MovieReleaseName	MovieFPS	SeriesSeason	SeriesEpisode	SeriesIMDBParent	MovieKind	URL
Line 2: 1	Alien 3	1992	English	en	2004-10-31 23:54:23	103644	sub	2	 Alien.3	11.000	0	0	0	movie	http://www.opensubtitles.org/subtitles/1/alien-3-en
Line 3: 2	Identity	2003	Slovenian	sl	2004-10-31 23:54:23	309698	sub	1	 Identity.DVDRiP.XViD	0.000	0	0	0	movie	http://www.opensubtitles.org/subtitles/2/identity-sl
Line 4: 3	Ghost in the Shell 2: Innocence	2004	English	en	2004-10-31 23:54:23	347246	srt	1	 Innocence.2004.DVDRip.XviD	23.980	0	0	0	movie	http://www.opensubtitles.org/subtitles/3/ghost-in-the-shell-2-innocence-en
Line 5: 4	Planet of the Apes	2001	Slovenian	sl	2004-10-31 23:54:23	133152	sub	1	 Planet.Of.The. Apes.XviD	0.000	0	0	0	movie	http://www.opensubtitles.org/subtitles/4/planet-of-the-apes-sl
Line 6: 5	The City of Lost Children	1995	English	en	2004-11-01 

In [6]:
import pandas as pd
import gzip

metadata_path = "subtitles_all.txt.gz"
correct_separator = '\t' # Confirmed from your previous output

print("--- Running Step 2: Loading Metadata ---")
try:
    # Load the data into the variable named metadata_df
    metadata_df = pd.read_csv(
        metadata_path,
        compression='gzip',
        sep=correct_separator,
        low_memory=False
        # Add error handling like quoting=3 or on_bad_lines='skip' if needed,
        # but start with just the correct separator.
    )
    print("Metadata loaded successfully into 'metadata_df'.")
    print("Shape:", metadata_df.shape)
    print("Columns:", metadata_df.columns.tolist())
    print("First 3 rows:\n", metadata_df.head(3))

except FileNotFoundError:
     print(f"Error: The file '{metadata_path}' was not found. Check the path.")
except Exception as e:
    print(f"Error loading metadata: {e}")
    # If you still get errors here, inspect the error message.
    # It might be a quoting issue (try adding quoting=3 inside read_csv)
    # or specific problematic lines (try on_bad_lines='warn' or on_bad_lines='skip')

print("--- Finished Step 2 ---")

# ---- NOW YOU CAN RUN THE STEP 3 CODE ----
# (The code block you ran previously that started with 'import sqlite3...')
# Make sure 'metadata_df' now exists before proceeding.
if 'metadata_df' in locals():
     print("\n'metadata_df' exists. Proceeding to run Step 3 code...")
     # PASTE THE ENTIRE STEP 3 CODE HERE or run the cell containing it
else:
     print("\nError: 'metadata_df' still not loaded. Fix Step 2 before running Step 3.")

--- Running Step 2: Loading Metadata ---
Error loading metadata: Error tokenizing data. C error: Expected 16 fields in line 2895, saw 17

--- Finished Step 2 ---

Error: 'metadata_df' still not loaded. Fix Step 2 before running Step 3.


In [5]:
import sqlite3
import pandas as pd # Make sure pandas is imported
import io
import zipfile
import random # To select random samples

# --- Prerequisites ---
# 1. Ensure 'metadata_df' is loaded correctly from Step 2, for example:
# metadata_path = "subtitles_all.txt.gz"
# metadata_df = pd.read_csv(metadata_path, compression='gzip', sep='\t', low_memory=False)
# print("Metadata loaded successfully. Shape:", metadata_df.shape)

# 2. Define database path
db_path = "opensubs.db"

# 3. Define the linking columns (based on inspection)
id_column_in_metadata = 'IDSubtitle' # From subtitles_all.txt.gz header
id_column_in_db = 'num'            # Primary key from subz table schema

# 4. Check if metadata DataFrame exists and has the ID column
if 'metadata_df' not in locals():
    print("Error: metadata_df does not exist. Please load it first (Step 2).")
elif id_column_in_metadata not in metadata_df.columns:
    print(f"Error: Column '{id_column_in_metadata}' not found in metadata_df.")
else:
    print("Proceeding with Step 3: Connecting Metadata and Subtitle Content...")

    # --- Sampling ---
    sample_size = 15 # How many subtitles to sample and extract
    try:
        # Get a list of valid IDs from the metadata, drop missing values, ensure unique
        valid_ids = metadata_df[id_column_in_metadata].dropna().unique()
        # Select random sample IDs
        if len(valid_ids) < sample_size:
             print(f"Warning: Requested sample size {sample_size} is larger than available unique valid IDs {len(valid_ids)}. Using all valid IDs.")
             sample_ids = valid_ids.tolist()
        else:
             sample_ids = random.sample(list(valid_ids), sample_size)

        # Convert IDs to integers for the SQL query as 'num' is INTEGER
        sample_ids_int = [int(id) for id in sample_ids]
        print(f"\nSelected sample IDs: {sample_ids_int}")

    except Exception as e:
        print(f"Error during sampling: {e}")
        sample_ids_int = [] # Prevent further execution if sampling fails

    # --- Database Connection and Querying ---
    if sample_ids_int: # Only proceed if we have IDs to query
        conn = None # Initialize conn to None for finally block
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            # Prepare the query to fetch BLOBs for these IDs
            placeholders = ','.join('?' * len(sample_ids_int))
            # Select the ID, name hint, and the blob data
            query = f"SELECT {id_column_in_db}, name, file FROM subz WHERE {id_column_in_db} IN ({placeholders})"

            print(f"\nExecuting query: {query.replace('?', '%s') % tuple(sample_ids_int)}") # Show query for debugging

            # Fetch the data
            cursor.execute(query, sample_ids_int)
            sample_rows = cursor.fetchall()
            print(f"\nFetched {len(sample_rows)} rows from the database for the sampled IDs.")

            # --- Process Each Fetched Row (Extract and Analyze) ---
            for row in sample_rows:
                sub_id, name_field, blob_data = row
                print(f"\n--- Processing DB ID (num): {sub_id}, Filename hint: {name_field} ---")

                if not blob_data:
                    print("  Skipping: No blob data found in database for this ID.")
                    continue

                try:
                    # Use io.BytesIO to treat the blob data like an in-memory file
                    zip_buffer = io.BytesIO(blob_data)

                    # Open the zip archive using a 'with' statement
                    with zipfile.ZipFile(zip_buffer, 'r') as zip_ref:
                        file_list = zip_ref.namelist()
                        print(f"  Files inside ZIP: {file_list}")

                        if not file_list:
                            print("  Skipping: ZIP archive is empty.")
                            continue

                        # Attempt to read the first file in the archive
                        subtitle_filename = file_list[0]
                        print(f"  Attempting to read: {subtitle_filename}")

                        # Open the file within the zip archive
                        with zip_ref.open(subtitle_filename) as subtitle_file:
                            # Read the raw bytes first
                            raw_content = subtitle_file.read()

                            # Attempt to decode using common encodings
                            content = None
                            encodings_to_try = ['utf-8', 'latin-1', 'cp1252'] # Add more if needed
                            for enc in encodings_to_try:
                                try:
                                    content = raw_content.decode(enc)
                                    print(f"  Successfully decoded with '{enc}'.")
                                    break # Stop trying once successful
                                except UnicodeDecodeError:
                                    continue # Try the next encoding
                                except Exception as decode_err:
                                    print(f"  Error during decoding with {enc}: {decode_err}")
                                    break # Stop if other error occurs

                            if content:
                                print(f"  First 200 chars:\n{content[:200].replace(chr(13),' ')}...") # Replace CR sometimes helps display
                                # --- Basic analysis on 'content' ---
                                line_count = len(content.splitlines())
                                word_count = len(content.split()) # Simple split by whitespace
                                print(f"  Analysis: Line count approx: {line_count}, Word count approx: {word_count}")
                            else:
                                print("  Failed to decode subtitle content with tried encodings.")

                except zipfile.BadZipFile:
                    print("  Skipping: Invalid or corrupted ZIP file.")
                except IndexError:
                     print("  Skipping: Could not access file list (IndexError).")
                except Exception as e:
                    print(f"  An error occurred processing blob for ID {sub_id}: {e}")

        except sqlite3.Error as sql_err:
            print(f"Database error: {sql_err}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
        finally:
            # Ensure the database connection is closed
            if conn:
                conn.close()
                print("\nDatabase connection closed.")
    else:
         print("\nSkipping database query because no valid sample IDs were generated.")

Error: metadata_df does not exist. Please load it first (Step 2).


In [7]:
import gzip

metadata_path = "subtitles_all.txt.gz"
target_line = 2895
context = 3 # Show a few lines before and after

print(f"--- Inspecting lines around {target_line} in {metadata_path} ---")
try:
    with gzip.open(metadata_path, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            line_num = i + 1
            if target_line - context <= line_num <= target_line + context:
                print(f"Line {line_num}: {line.strip()}")
            if line_num > target_line + context:
                break
except Exception as e:
    print(f"Error reading file: {e}")
    # Consider trying other encodings like 'latin-1' if utf-8 fails
print("--- End of file inspection ---")

--- Inspecting lines around 2895 in subtitles_all.txt.gz ---
Line 2892: 2481	In Harm's Way	1965	English	en	2005-03-01 00:00:00	59309	srt	2	In Harm's Way	0.000	0	0	0	movie	http://www.opensubtitles.org/subtitles/2481/in-harm-s-way-en
Line 2893: 2482	Bunny Lake Is Missing	1965	English	en	2005-03-01 00:00:00	58997	srt	1	Bunny Lake Is Missing	0.000	0	0	0	movie	http://www.opensubtitles.org/subtitles/2482/bunny-lake-is-missing-en
Line 2894: 2483	Dark Habits	1983	English	en	2005-03-01 00:00:00	85496	srt	1	Entre tinieblas (Dark Habits)	24.999	0	0	0	movie	http://www.opensubtitles.org/subtitles/2483/dark-habits-en
Line 2895: 2484	What Have I Done to Deserve This?	1984	English	en	2005-03-01 00:00:00	88461	srt	1	¿Qué he hecho yo para merecer esto!! Poster 	 SEE RANK ¿Qué he hecho yo para merecer esto!!	25.000	0	0	0	movie	http://www.opensubtitles.org/subtitles/2484/what-have-i-done-to-deserve-this-en
Line 2896: 6448582	"Downton Abbey" Episode #3.6	2012	Polish	pl	2016-01-04 13:28:13	2385246	sub	1	 Do

In [3]:
import pandas as pd
import gzip
import csv # Still potentially useful for quoting=csv.QUOTE_NONE

metadata_path = "subtitles_all.txt.gz"
correct_separator = '\t'

print("--- Running Step 2: Loading Metadata (Attempt 3 - Skipping Errors) ---")
try:
    metadata_df = pd.read_csv(
        metadata_path,
        compression='gzip',
        sep=correct_separator,
        quoting=csv.QUOTE_NONE, # Keep this, might help other lines
        on_bad_lines='skip',   # <--- ADD THIS LINE TO SKIP BAD LINES
        low_memory=False
    )
    # If loading succeeds, Pandas might issue warnings about skipped lines, which is expected.
    print("Metadata loaded successfully into 'metadata_df' (potentially with skipped rows).")
    print("Shape:", metadata_df.shape) # Check how many rows were loaded vs expected (~5.7 million)
    print("Columns:", metadata_df.columns.tolist())
    print("First 3 rows:\n", metadata_df.head(3))

except FileNotFoundError:
     print(f"Error: The file '{metadata_path}' was not found. Check the path.")
except Exception as e:
    # Catching other potential critical errors during load
    print(f"Error loading metadata even with skipping bad lines: {e}")

print("--- Finished Step 2 ---")

# Check if metadata_df was loaded before trying Step 3
if 'metadata_df' in locals():
     print(f"\n'metadata_df' exists. Proceeding to run Step 3 code...")
     # You can now run your Step 3 code again.
     # PASTE THE ENTIRE STEP 3 CODE HERE or run the cell containing it
else:
     print("\nError: 'metadata_df' still not loaded. Loading failed even with on_bad_lines='skip'. Check previous errors.")

--- Running Step 2: Loading Metadata (Attempt 3 - Skipping Errors) ---
Metadata loaded successfully into 'metadata_df' (potentially with skipped rows).
Shape: (5729550, 16)
Columns: ['IDSubtitle', 'MovieName', 'MovieYear', 'LanguageName', 'ISO639', 'SubAddDate', 'ImdbID', 'SubFormat', 'SubSumCD', 'MovieReleaseName', 'MovieFPS', 'SeriesSeason', 'SeriesEpisode', 'SeriesIMDBParent', 'MovieKind', 'URL']
First 3 rows:
   IDSubtitle                        MovieName  MovieYear LanguageName ISO639  \
0          1                          Alien 3     1992.0      English     en   
1          2                         Identity     2003.0    Slovenian     sl   
2          3  Ghost in the Shell 2: Innocence     2004.0      English     en   

            SubAddDate  ImdbID SubFormat  SubSumCD  \
0  2004-10-31 23:54:23  103644       sub       2.0   
1  2004-10-31 23:54:23  309698       sub       1.0   
2  2004-10-31 23:54:23  347246       srt       1.0   

              MovieReleaseName  MovieFPS  Se

In [4]:
import sqlite3
import pandas as pd # Make sure pandas is imported if in a new cell/script
import io
import zipfile
import random # To select random samples

# --- Prerequisites (Assumed Met) ---
# 1. 'metadata_df' exists from Step 2.
# 2. Database path is correct.

db_path = "opensubs.db"

# Define the linking columns (based on inspection)
id_column_in_metadata = 'IDSubtitle' # From subtitles_all.txt.gz header
id_column_in_db = 'num'            # Primary key from subz table schema

# Check again just in case (good practice)
if 'metadata_df' not in locals():
    print("Error: metadata_df does not exist. Please load it first (Step 2).")
elif id_column_in_metadata not in metadata_df.columns:
    print(f"Error: Column '{id_column_in_metadata}' not found in metadata_df.")
else:
    print("--- Running Step 3: Connect Metadata and Subtitle Content (Sampling) ---")

    # --- Sampling ---
    sample_size = 15 # How many subtitles to sample and extract
    try:
        # Get a list of valid IDs from the metadata, drop missing values, ensure unique
        valid_ids = metadata_df[id_column_in_metadata].dropna().unique()
        # Select random sample IDs
        if len(valid_ids) == 0:
             print("Error: No valid IDs found in the metadata column '{id_column_in_metadata}'. Cannot sample.")
             sample_ids_int = []
        elif len(valid_ids) < sample_size:
             print(f"Warning: Requested sample size {sample_size} is larger than available unique valid IDs {len(valid_ids)}. Using all valid IDs.")
             sample_ids = valid_ids.tolist()
             # Convert IDs to integers for the SQL query as 'num' is INTEGER
             sample_ids_int = [int(id) for id in sample_ids]
        else:
             sample_ids = random.sample(list(valid_ids), sample_size)
             # Convert IDs to integers for the SQL query as 'num' is INTEGER
             sample_ids_int = [int(id) for id in sample_ids]

        if sample_ids_int:
             print(f"\nSelected sample IDs: {sample_ids_int}")

    except Exception as e:
        print(f"Error during sampling: {e}")
        sample_ids_int = [] # Prevent further execution if sampling fails

    # --- Database Connection and Querying ---
    if sample_ids_int: # Only proceed if we have IDs to query
        conn = None # Initialize conn to None for finally block
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            # Prepare the query to fetch BLOBs for these IDs
            placeholders = ','.join('?' * len(sample_ids_int))
            # Select the ID, name hint, and the blob data
            query = f"SELECT {id_column_in_db}, name, file FROM subz WHERE {id_column_in_db} IN ({placeholders})"

            # Fetch the data
            cursor.execute(query, sample_ids_int)
            sample_rows = cursor.fetchall()
            print(f"\nFetched {len(sample_rows)} rows from the database for the {len(sample_ids_int)} sampled IDs.")
            if len(sample_rows) < len(sample_ids_int):
                 print("  (Note: Some sampled IDs might not exist in the database)")


            # --- Process Each Fetched Row (Extract and Analyze) ---
            for row in sample_rows:
                sub_id, name_field, blob_data = row
                print(f"\n--- Processing DB ID (num): {sub_id}, Filename hint: {name_field} ---")

                if not blob_data:
                    print("  Skipping: No blob data found in database for this ID.")
                    continue

                try:
                    # Use io.BytesIO to treat the blob data like an in-memory file
                    zip_buffer = io.BytesIO(blob_data)

                    # Open the zip archive using a 'with' statement
                    with zipfile.ZipFile(zip_buffer, 'r') as zip_ref:
                        file_list = zip_ref.namelist()
                        print(f"  Files inside ZIP: {file_list}")

                        if not file_list:
                            print("  Skipping: ZIP archive is empty.")
                            continue

                        # Attempt to read the first file in the archive
                        subtitle_filename = file_list[0]
                        print(f"  Attempting to read: {subtitle_filename}")

                        # Open the file within the zip archive
                        with zip_ref.open(subtitle_filename) as subtitle_file:
                            # Read the raw bytes first
                            raw_content = subtitle_file.read()

                            # Attempt to decode using common encodings
                            content = None
                            encodings_to_try = ['utf-8', 'latin-1', 'cp1252'] # Add more if needed
                            for enc in encodings_to_try:
                                try:
                                    content = raw_content.decode(enc)
                                    print(f"  Successfully decoded with '{enc}'.")
                                    break # Stop trying once successful
                                except UnicodeDecodeError:
                                    continue # Try the next encoding
                                except Exception as decode_err:
                                    print(f"  Error during decoding with {enc}: {decode_err}")
                                    break # Stop if other error occurs

                            if content:
                                # Replace carriage returns for potentially cleaner printing
                                clean_content_sample = content[:200].replace('\r', ' ')
                                print(f"  First 200 chars:\n{clean_content_sample}...")
                                # --- Basic analysis on 'content' ---
                                line_count = len(content.splitlines())
                                word_count = len(content.split()) # Simple split by whitespace
                                print(f"  Analysis: Line count approx: {line_count}, Word count approx: {word_count}")
                            else:
                                print("  Failed to decode subtitle content with tried encodings.")

                except zipfile.BadZipFile:
                    print("  Skipping: Invalid or corrupted ZIP file.")
                except IndexError:
                     print("  Skipping: Could not access file list (IndexError).")
                except Exception as e:
                    print(f"  An error occurred processing blob for ID {sub_id}: {e}")

        except sqlite3.Error as sql_err:
            print(f"Database error: {sql_err}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
        finally:
            # Ensure the database connection is closed
            if conn:
                conn.close()
                print("\nDatabase connection closed.")
    else:
         print("\nSkipping database query because no valid sample IDs were generated.")

    print("\n--- Finished Step 3 ---")

--- Running Step 3: Connect Metadata and Subtitle Content (Sampling) ---

Selected sample IDs: [4896472, 3880466, 8802174, 8380138, 7188275, 4496982, 5588367, 3886858, 5401298, 4036802, 7006770, 6301994, 4155385, 3414835, 8299439]

Fetched 15 rows from the database for the 15 sampled IDs.

--- Processing DB ID (num): 3414835, Filename hint: attachment; filename="millers.crossing.(1990).pol.1cd.(3414835).zip" ---
  Files inside ZIP: ["01.1990.Miller's.Crossing.movie.dvdrip.avi.hokusbloke.txt", 'miller.s.crossing.(3414835).nfo']
  Attempting to read: 01.1990.Miller's.Crossing.movie.dvdrip.avi.hokusbloke.txt
  Successfully decoded with 'latin-1'.
  First 200 chars:
{1}{100}www.napiprojekt.pl - nowa jakoæ napisów.|Napisy zosta³y specjalnie dopasowane do Twojej wersji filmu. 
{125}{275}-== [ www.OpenSubtitles.org ] ==- 
{710}{778}Mówiê o przyjani. 
{780}{860}Mów...
  Analysis: Line count approx: 1347, Word count approx: 7978

--- Processing DB ID (num): 3880466, Filename hint: attachment;

In [5]:
# Example using pysrt (inside your Step 3 loop after getting 'content')
import pysrt
import io

try:
    subs = pysrt.from_string(content)
    dialogue_only = subs.text # Get dialogue concatenated
    print(f"  Parsed dialogue (first 200 chars): {dialogue_only[:200]}...")
    # You can now iterate through subs: for sub in subs: print(sub.start, sub.end, sub.text)
except Exception as parse_err:
    print(f"  Could not parse SRT structure: {parse_err}")

  Parsed dialogue (first 200 chars): Who are the real-world Illuminati ?
Find out @ saveanilluminati.com
<i>April 1865.</i>
<i>Dear Ma and Pa, I hope
this letter finds you well.</i>
<i>I have some good news.</i>
<i>Word is the war
may be...


In [6]:
# Example filtering metadata_df BEFORE sampling in Step 3
metadata_df_filtered = metadata_df[
    (metadata_df['LanguageName'] == 'English') &
    (metadata_df['MovieKind'] == 'movie') &
    (metadata_df['MovieYear'] >= 2010)
].copy()
# Then sample IDs from metadata_df_filtered instead of metadata_df

In [7]:
metadata_df_filtered

,IDSubtitle,MovieName,MovieYear,LanguageName,ISO639,SubAddDate,ImdbID,SubFormat,SubSumCD,MovieReleaseName,MovieFPS,SeriesSeason,SeriesEpisode,SeriesIMDBParent,MovieKind,URL
248,4465952,The Rum Diary,2011.0,English,en,2012-02-09 00:16:20,376136,srt,1.0,The.Rum.Diary.2011.1080p.MKV.X264.AC3.DTS.Eng.NL,23.976,0.0,0.0,0.0,movie,http://www.opensubtitles.org/subtitles/4465952...
364,4788064,Sen Kimsin?,2012.0,English,en,2013-01-28 00:17:10,2142883,srt,1.0,Sen kimsin (2012).eng.DP,25.000,0.0,0.0,0.0,movie,http://www.opensubtitles.org/subtitles/4788064...
574,4490619,Abduction,2011.0,English,en,2012-03-15 00:18:19,1600195,srt,1.0,Abduction.2011.BluRay.1080p.DTS.x264-CHD,23.976,0.0,0.0,0.0,movie,http://www.opensubtitles.org/subtitles/4490619...
756,7699952,Hana wa sakuka,2018.0,English,en,2019-03-27 14:44:41,7408772,srt,1.0,Hana wa Saku ka (Does The Flower Bloom),29.970,0.0,0.0,0.0,movie,http://www.opensubtitles.org/subtitles/7699952...
891,4550013,Tom Sawyer,2011.0,English,en,2012-05-15 00:19:36,1722484,srt,1.0,5rFF-toms480p,24.000,0.0,0.0,0.0,movie,http://www.opensubtitles.org/subtitles/4550013...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5729436,9179986,Jug Jugg Jeeyo,2022.0,English,en,2022-07-24 06:23:45,13449624,srt,1.0,JugJugg.Jeeyo.2022.1080p.AMZN.WEB-DL.Hindi.DD...,0.000,0.0,0.0,0.0,movie,http://www.opensubtitles.org/subtitles/9179986...
5729489,9180049,Detroit Dreams,2022.0,English,en,2022-07-24 09:05:40,15100080,srt,1.0,WEBRip.800MB.x264-GalaxyRG,23.976,0.0,0.0,0.0,movie,http://www.opensubtitles.org/subtitles/9180049...
5729490,9180050,Detroit Dreams,2022.0,English,en,2022-07-24 09:06:27,15100080,srt,1.0,WEBRip.800MB.x264-GalaxyRG,23.976,0.0,0.0,0.0,movie,http://www.opensubtitles.org/subtitles/9180050...
5729493,9180053,Hope Gap,2019.0,English,en,2022-07-24 09:32:19,7587876,srt,1.0,Hope.Gap.2019.720p.BluRay.x264.AAC-[YTS.MX],24.000,0.0,0.0,0.0,movie,http://www.opensubtitles.org/subtitles/9180053...


In [8]:
import sqlite3
import pandas as pd
import io
import zipfile
import json
import os
import pysrt # Make sure pysrt is installed: pip install pysrt
from tqdm import tqdm # For progress bar: pip install tqdm
import time
import random
import csv # For quoting=csv.QUOTE_NONE

# --- Configuration ---
DB_PATH = "opensubs.db"
METADATA_PATH = "subtitles_all.txt.gz"
OUTPUT_JSONL_FILE = "extracted_subtitles_200_movies.jsonl"
ERROR_LOG_FILE = "extraction_errors_200_movies.log"
ENCODINGS_TO_TRY = ['utf-8', 'latin-1', 'cp1252']

NUM_MOVIES_TO_SAMPLE = 200
# Assuming 'ImdbID' reliably identifies a unique movie or TV show parent
MOVIE_ID_COLUMN = 'ImdbID'
SUBTITLE_ID_COLUMN = 'IDSubtitle' # Corresponds to 'num' in DB

# --- Functions (Keep the same as before) ---
def process_db_row(row_data, output_f, error_f):
    """
    Processes a single row from the database, extracts subtitle text,
    and writes it to the output file or logs errors.
    (Identical to the function in the 'extract all' script)
    """
    sub_id, blob_data = row_data
    result = {"id": sub_id, "dialogue": None, "raw_srt": None, "error": None}
    srt_filename = None
    content = None
    decoded = False

    if not blob_data:
        result["error"] = "No blob data"
        log_error(error_f, sub_id, "No blob data")
        output_f.write(json.dumps(result) + '\n')
        return

    try:
        zip_buffer = io.BytesIO(blob_data)
        with zipfile.ZipFile(zip_buffer, 'r') as zip_ref:
            file_list = zip_ref.namelist()
            srt_files = [f for f in file_list if f.lower().endswith('.srt')]
            if srt_files:
                srt_filename = srt_files[0]
            elif file_list:
                 srt_filename = file_list[0]
            else:
                 result["error"] = "ZIP archive is empty"
                 log_error(error_f, sub_id, "ZIP archive is empty")
                 output_f.write(json.dumps(result) + '\n')
                 return

            with zip_ref.open(srt_filename) as subtitle_file:
                raw_content = subtitle_file.read()

            for enc in ENCODINGS_TO_TRY:
                try:
                    content = raw_content.decode(enc)
                    decoded = True
                    break
                except UnicodeDecodeError:
                    continue
                except Exception as decode_err:
                     log_error(error_f, sub_id, f"Unexpected decode error with {enc} on file {srt_filename}: {decode_err}")

            if not decoded:
                result["error"] = f"Failed to decode file {srt_filename} with tried encodings."
                log_error(error_f, sub_id, result["error"])
                output_f.write(json.dumps(result) + '\n')
                return

            result["raw_srt"] = content

            try:
                subs = pysrt.from_string(content)
                result["dialogue"] = subs.text
            except Exception as parse_err:
                result["error"] = f"SRT parsing error: {parse_err}"
                log_error(error_f, sub_id, f"SRT parsing error on file {srt_filename}: {parse_err}")

    except zipfile.BadZipFile:
        result["error"] = "Invalid ZIP file"
        log_error(error_f, sub_id, "Invalid ZIP file")
    except IndexError:
         result["error"] = "Error accessing file list in ZIP"
         log_error(error_f, sub_id, "Error accessing file list in ZIP")
    except Exception as e:
        result["error"] = f"Unexpected processing error: {e}"
        log_error(error_f, sub_id, f"Unexpected processing error: {e}")

    output_f.write(json.dumps(result) + '\n')

def log_error(error_f, sub_id, message):
    """Appends an error message to the log file."""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    error_f.write(f"{timestamp} | ID: {sub_id} | Error: {message}\n")

# --- Main Execution ---
if __name__ == "__main__":
    print(f"Starting extraction for {NUM_MOVIES_TO_SAMPLE} movies...")
    start_time = time.time()

    metadata_df = None
    subtitle_ids_to_fetch = []

    # --- Step 1: Load Metadata ---
    print(f"\nLoading metadata from '{METADATA_PATH}'...")
    try:
        metadata_df = pd.read_csv(
            METADATA_PATH,
            compression='gzip',
            sep='\t', # Use Tab separator
            quoting=csv.QUOTE_NONE, # Ignore quoting issues
            on_bad_lines='skip', # Skip problematic lines
            low_memory=False
        )
        print(f"Metadata loaded successfully. Shape: {metadata_df.shape}")
        # Check if essential columns exist
        if MOVIE_ID_COLUMN not in metadata_df.columns or SUBTITLE_ID_COLUMN not in metadata_df.columns:
             raise ValueError(f"Required columns ('{MOVIE_ID_COLUMN}', '{SUBTITLE_ID_COLUMN}') not found in metadata.")

    except FileNotFoundError:
         print(f"Error: Metadata file '{METADATA_PATH}' not found.")
         exit()
    except Exception as e:
        print(f"Error loading metadata: {e}")
        exit()

    # --- Step 2: Select 200 Unique Movies ---
    print(f"\nSelecting {NUM_MOVIES_TO_SAMPLE} unique movies based on '{MOVIE_ID_COLUMN}'...")
    try:
        unique_movie_ids = metadata_df[MOVIE_ID_COLUMN].dropna().unique()
        if len(unique_movie_ids) < NUM_MOVIES_TO_SAMPLE:
            print(f"Warning: Found only {len(unique_movie_ids)} unique movie IDs, less than requested {NUM_MOVIES_TO_SAMPLE}. Using all available.")
            sampled_movie_ids = list(unique_movie_ids)
        else:
            sampled_movie_ids = random.sample(list(unique_movie_ids), NUM_MOVIES_TO_SAMPLE)
        print(f"Selected {len(sampled_movie_ids)} movie IDs.")

    except Exception as e:
        print(f"Error during movie sampling: {e}")
        exit()

    # --- Step 3: Find All Subtitle IDs for Selected Movies ---
    print(f"\nFinding subtitle IDs ({SUBTITLE_ID_COLUMN}) for the selected movies...")
    try:
        # Filter metadata for the selected movies
        filtered_metadata = metadata_df[metadata_df[MOVIE_ID_COLUMN].isin(sampled_movie_ids)]
        # Get the corresponding subtitle IDs and ensure they are integers
        subtitle_ids_to_fetch = filtered_metadata[SUBTITLE_ID_COLUMN].dropna().astype(int).unique().tolist()
        print(f"Found {len(subtitle_ids_to_fetch):,} subtitle files associated with the {len(sampled_movie_ids)} selected movies.")

    except Exception as e:
        print(f"Error finding subtitle IDs: {e}")
        exit()

    # --- Step 4: Fetch and Extract Subtitles from DB ---
    if not subtitle_ids_to_fetch:
        print("\nNo subtitle IDs found for the selected movies. Exiting.")
        exit()

    print(f"\nConnecting to database '{DB_PATH}' to extract {len(subtitle_ids_to_fetch)} subtitles...")
    print(f"Output will be written to '{OUTPUT_JSONL_FILE}'")
    print(f"Errors will be logged in '{ERROR_LOG_FILE}'")
    print("-" * 30)

    conn = None
    output_file = None
    error_file = None

    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Open files for appending
        output_file = open(OUTPUT_JSONL_FILE, 'a', encoding='utf-8')
        error_file = open(ERROR_LOG_FILE, 'a', encoding='utf-8')

        # Fetch subtitles in batches to avoid huge SQL query strings (optional but good practice)
        batch_size = 500 # How many IDs to query at once
        num_batches = (len(subtitle_ids_to_fetch) + batch_size - 1) // batch_size
        print(f"Processing in {num_batches} batches of up to {batch_size} IDs...")

        processed_count = 0
        with tqdm(total=len(subtitle_ids_to_fetch), unit="subs") as pbar:
            for i in range(0, len(subtitle_ids_to_fetch), batch_size):
                batch_ids = subtitle_ids_to_fetch[i:i+batch_size]
                placeholders = ','.join('?' * len(batch_ids))
                query = f"SELECT num, file FROM subz WHERE num IN ({placeholders})"

                try:
                    cursor.execute(query, batch_ids)
                    rows = cursor.fetchall()
                    for row in rows:
                         process_db_row(row, output_file, error_file)
                         processed_count += 1
                         pbar.update(1) # Update progress bar for each row processed
                except sqlite3.Error as batch_sql_err:
                     log_error(error_file, f"Batch {i//batch_size + 1}", f"Database error during batch query: {batch_sql_err}")
                     pbar.update(len(batch_ids) - cursor.rowcount) # Try to advance progress bar past failed batch

            pbar.close() # Ensure tqdm finishes cleanly if loop finishes early

        print(f"\nProcessed {processed_count} subtitles from the database.")
        if processed_count < len(subtitle_ids_to_fetch):
             print(f"Warning: Expected {len(subtitle_ids_to_fetch)} subtitles based on metadata, but only processed {processed_count} from DB. Some IDs might be missing in the DB.")


    except sqlite3.Error as sql_err:
        print(f"\nDatabase error: {sql_err}")
        if error_file: log_error(error_file, "N/A", f"Database error: {sql_err}")
    except IOError as io_err:
        print(f"\nFile I/O error: {io_err}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        if error_file: log_error(error_file, "N/A", f"Unexpected error: {e}")
    finally:
        # Ensure resources are closed
        # (Cursor closing is handled implicitly when connection closes if not iterated fully)
        if conn:
            conn.close()
            print("Database connection closed.")
        if output_file:
            output_file.close()
            print(f"Output file '{OUTPUT_JSONL_FILE}' closed.")
        if error_file:
            error_file.close()
            print(f"Error log '{ERROR_LOG_FILE}' closed.")

        end_time = time.time()
        duration = end_time - start_time
        print(f"Extraction finished in {duration:.2f} seconds ({duration/60:.2f} minutes).")

Starting extraction for 200 movies...

Loading metadata from 'subtitles_all.txt.gz'...
Metadata loaded successfully. Shape: (5729550, 16)

Selecting 200 unique movies based on 'ImdbID'...
Selected 200 movie IDs.

Finding subtitle IDs (IDSubtitle) for the selected movies...
Found 2,403 subtitle files associated with the 200 selected movies.

Connecting to database 'opensubs.db' to extract 2403 subtitles...
Output will be written to 'extracted_subtitles_200_movies.jsonl'
Errors will be logged in 'extraction_errors_200_movies.log'
------------------------------
Processing in 5 batches of up to 500 IDs...


100%|█████████▉| 2396/2403 [00:58<00:00, 41.15subs/s] 


Processed 2396 subtitles from the database.
Database connection closed.
Output file 'extracted_subtitles_200_movies.jsonl' closed.
Error log 'extraction_errors_200_movies.log' closed.
Extraction finished in 83.45 seconds (1.39 minutes).


In [1]:
import json
import csv
import os

# --- Configuration ---
input_jsonl_file = 'extracted_subtitles_200_movies.jsonl'
output_csv_file = 'extracted_subtitles_200_movies.csv'
# Define the columns you want in your CSV file (matching keys in the JSON)
fieldnames = ['id', 'dialogue']
# --- Configuration End ---

# Basic check if input file exists
if not os.path.exists(input_jsonl_file):
    print(f"Error: Input file not found at '{input_jsonl_file}'")
else:
    print(f"Starting conversion from '{input_jsonl_file}' to '{output_csv_file}'...")
    lines_processed = 0
    rows_written = 0
    errors_encountered = 0

    try:
        # Open the input JSONL file for reading with UTF-8 encoding
        # Open the output CSV file for writing with UTF-8 encoding
        # newline='' is important to prevent extra blank rows in the CSV
        with open(input_jsonl_file, 'r', encoding='utf-8') as infile, \
             open(output_csv_file, 'w', newline='', encoding='utf-8') as outfile:

            # Create a CSV writer object.
            # csv.QUOTE_MINIMAL tells it to only quote fields containing special characters
            # (like the delimiter (,), quote character ("), or line breaks).
            # Use csv.QUOTE_ALL to quote every field if preferred.
            writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)

            # Write the header row to the CSV file
            writer.writerow(fieldnames)

            # Iterate through each line in the JSONL file
            for line in infile:
                lines_processed += 1
                # Skip potentially empty lines
                if not line.strip():
                    continue

                try:
                    # Parse the JSON string on the current line into a Python dictionary
                    data = json.loads(line)

                    # Extract the data for the specified fields in the correct order
                    # Using .get(field) is safer than data[field] as it returns None
                    # if a key is missing, preventing a KeyError.
                    row_data = [data.get(field) for field in fieldnames]

                    # Write the extracted data as a row in the CSV file
                    writer.writerow(row_data)
                    rows_written += 1

                except json.JSONDecodeError:
                    errors_encountered += 1
                    print(f"Warning: Skipping line {lines_processed} due to invalid JSON: {line.strip()}")
                except Exception as e:
                    errors_encountered += 1
                    print(f"Warning: Skipping line {lines_processed} due to error: {e}")

        print("\nConversion Complete.")
        print(f"Total lines read: {lines_processed}")
        print(f"Rows written to CSV (excluding header): {rows_written}")
        if errors_encountered > 0:
            print(f"Lines skipped due to errors: {errors_encountered}")
        print(f"Output saved to: '{output_csv_file}'")

    except FileNotFoundError:
        # This check is technically redundant due to the os.path.exists above,
        # but good practice in case the file disappears between check and open.
        print(f"Error: Input file disappeared before opening: '{input_jsonl_file}'")
    except IOError as e:
        print(f"Error reading or writing file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Starting conversion from 'extracted_subtitles_200_movies.jsonl' to 'extracted_subtitles_200_movies.csv'...

Conversion Complete.
Total lines read: 2396
Rows written to CSV (excluding header): 2395
Lines skipped due to errors: 1
Output saved to: 'extracted_subtitles_200_movies.csv'
